In [3]:
import numpy as np

In [135]:
class DeepNN():
    def __init__(self, X, Y, dims_of_layers, activations, alpha = 0.01):
        #dims_of_layers - list of number of units in each layer (first element - num of features in input)
        #activations - activation function applied to each layer
        
        #dims_of_layers[0] - n of features in input
        #activations[0] - activation for first hidden layer
        #we support only 3 activation funcs: linear, sigmoid, relu
        
        #X.shape should be (n_features, m_examples)
        #Y.shape should be (1, m_examples)
        
        
        self.dims_of_layers = dims_of_layers
        self.n_layers = len(activations)
        
        self.activations = activations
        self.params = dict()
        
        self.learning_history = []
        self.alpha = alpha
        
        #setting cache dicts for backpropogation
        
        self.cache = dict()
        
    
    def initialize_params(self):
        
        for i in range(1, len(self.activations) + 1):
            
            #setting parameters layer by layer
            self.params["W" + str(i)] = np.random.randn(self.dims_of_layers[i], self.dims_of_layers[i-1])
            self.params["b" + str(i)] = np.zeros((self.dims_of_layers[i], 1))
            
     
    
    def activation(self, Z, function="linear"):
        if function == "linear":
            return Z
        
        if function == "sigmoid":
            return 1 / (1 + np.exp(-Z))
        
        if function == "relu":
            return Z * (Z > 0)
    
    def forward_propogation(self, X):
        
        #X.shape = (n_features, m_examples)
        
        A_prev = X
        
        Z_current = np.dot(self.params["W1"], A_prev) + self.params["b1"]
        A_current = self.activation(Z_current, function=self.activations[0])
        
        self.cache["Z1"] = Z_current
        self.cache["A1"] = A_current
        
        for i in range(1, len(self.activations)):
            A_prev = A_current
            
            #A_prev - cache["A" + str(i)]
            Z_current = np.dot(self.params["W" + str(i+1)], A_prev)
            A_current = self.activation(Z_current, function=self.activations[i])
            
            #keeping values in cache for backprop
            self.cache["Z" + str(i+1)] = Z_current
            self.cache["A" + str(i+1)] = A_current            

            
        predictions = A_current
        
        return predictions
        
        
            

In [131]:
dims = [3, 4, 2, 1]
activation = ["relu", "relu", "sigmoid"]


In [132]:
deepnn = DeepNN(dims, activation)
deepnn.initialize_params()

In [136]:
x = np.array([10, 5, 9]).reshape(-1, 1)

deepnn.forward_propogation(X)

array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]])

In [137]:
deepnn.cache

{'Z1': array([[-1.10303739, -1.10303739, -1.10303739, -1.10303739, -1.10303739,
         -1.10303739, -1.10303739, -1.10303739, -1.10303739, -1.10303739],
        [ 2.60756136,  2.60756136,  2.60756136,  2.60756136,  2.60756136,
          2.60756136,  2.60756136,  2.60756136,  2.60756136,  2.60756136],
        [ 1.2425657 ,  1.2425657 ,  1.2425657 ,  1.2425657 ,  1.2425657 ,
          1.2425657 ,  1.2425657 ,  1.2425657 ,  1.2425657 ,  1.2425657 ],
        [ 1.35633891,  1.35633891,  1.35633891,  1.35633891,  1.35633891,
          1.35633891,  1.35633891,  1.35633891,  1.35633891,  1.35633891]]),
 'A1': array([[-0.        , -0.        , -0.        , -0.        , -0.        ,
         -0.        , -0.        , -0.        , -0.        , -0.        ],
        [ 2.60756136,  2.60756136,  2.60756136,  2.60756136,  2.60756136,
          2.60756136,  2.60756136,  2.60756136,  2.60756136,  2.60756136],
        [ 1.2425657 ,  1.2425657 ,  1.2425657 ,  1.2425657 ,  1.2425657 ,
          1.242565